In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
N=3200

In [3]:
def spatial_reference(row, ref):
    d_lon = math.radians(float(row[0]) - float(ref[0]))
    d_lat = math.radians(float(row[1]) - float(ref[1]))
    a = math.sin(d_lat / 2) ** 2 + (
        math.cos(math.radians(float(ref[0])))
        * math.cos(math.radians(float(ref[0])))
        * math.sin(d_lon / 2) ** 2
    )
    d = 2.0 * 6371000.0 * math.asin(math.sqrt(a))
    phi, quadrant, orientation = 0.0, 0, 0
    if d > 0.0:
        phi = math.atan2(d_lon, d_lat) * 180.0 / math.pi
        if phi < 0.0:
            phi += 360.0
        quadrant = math.floor(phi / 90) + 1
        orientation = ["N", "E", "S", "W"][int((phi + 45.0) / 90.0) % 4]
    return pd.Series(
        {
            "direction": phi,
            "distance": d,
            "dz": float(row[2]) - float(ref[2]),
            "orientation": orientation,
            "quadrant": quadrant,
        }
    )


In [4]:
metadata = pd.DataFrame(np.random.randn(N,3))

In [7]:
metadata

,0,1,2
0,1.347183,0.016122,0.197940
1,-0.838429,-0.831782,0.586928
2,0.239215,-0.214303,0.806657
3,-1.311747,-1.594499,0.477434
4,-0.282448,-0.792031,-0.231877
...,...,...,...
3195,0.900206,0.373429,0.050427
3196,-0.949769,-0.654506,0.831882
3197,2.057285,-1.090179,0.545236
3198,-1.285892,0.234307,0.441431


In [12]:
%%timeit
ts_path = 0
ref = metadata.loc[ts_path]
metadata[["direction", "distance", "dz", "orientation", "quadrant"]] = metadata.apply(
            spatial_reference, args=(ref,), axis=1
        )

920 ms ± 19.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
metadata

,0,1,2,direction,distance,dz,orientation,quadrant
0,1.347183,0.016122,0.197940,0.000000,0.000000,0.000000,0,0.0
1,-0.838429,-0.831782,0.586928,248.796327,260618.138679,0.388988,W,3.0
2,0.239215,-0.214303,0.806657,258.251619,125803.325743,0.608717,W,3.0
3,-1.311747,-1.594499,0.477434,238.795013,345618.309253,0.279494,W,3.0
4,-0.282448,-0.792031,-0.231877,243.622639,202222.905613,-0.429816,W,3.0
...,...,...,...,...,...,...,...,...
3195,0.900206,0.373429,0.050427,308.638293,63619.341095,-0.147513,W,4.0
3196,-0.949769,-0.654506,0.831882,253.724075,266007.838794,0.633942,W,3.0
3197,2.057285,-1.090179,0.545236,147.304791,146165.314557,0.347296,S,2.0
3198,-1.285892,0.234307,0.441431,274.736883,293707.672148,0.243491,W,4.0


In [15]:

from scipy.spatial import distance_matrix

In [18]:

metadata = np.random.randn(N,3)


In [19]:
%%timeit
DIST = distance_matrix(metadata, metadata) + np.diag([np.inf] * N)
IDX = DIST.argsort()

840 ms ± 7.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
